In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2022-07-13 17:17:16,3.0707,172.7902,82.36,78,88,18.86
1,1,Bredasdorp,ZA,2022-07-13 17:17:16,-34.5322,20.0403,58.42,64,43,18.72
2,2,Kavaratti,IN,2022-07-13 17:17:17,10.5669,72.6420,81.36,83,99,19.15
3,3,Vaini,TO,2022-07-13 17:17:17,-21.2000,-175.2000,75.36,94,75,12.66
4,4,Ushuaia,AR,2022-07-13 17:17:17,-54.8000,-68.3000,38.82,75,75,10.36


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Bambous Virieux,MU,2022-07-13 17:17:19,-20.3428,57.7575,70.05,68,40,11.50
12,12,Sovetskiy,RU,2022-07-13 17:17:20,61.3614,63.5842,71.31,69,55,4.09
27,27,Rikitea,PF,2022-07-13 17:17:24,-23.1203,-134.9692,71.28,83,59,3.33
28,28,Ponta Do Sol,PT,2022-07-13 17:17:24,32.6667,-17.1000,74.12,66,3,6.35
46,46,Avarua,CK,2022-07-13 17:18:30,-21.2078,-159.7750,71.65,68,75,6.91
59,59,Souillac,MU,2022-07-13 17:18:33,-20.5167,57.5167,70.18,68,40,11.50
76,76,Melfi,IT,2022-07-13 17:18:37,40.9965,15.6514,70.50,25,0,12.75
79,79,Nouadhibou,MR,2022-07-13 17:18:38,20.9310,-17.0347,73.38,73,13,26.46
87,87,Chekhov,RU,2022-07-13 17:18:40,55.1431,37.4711,71.55,66,81,4.12
88,88,Koumac,NC,2022-07-13 17:18:41,-20.5667,164.2833,73.38,77,100,12.97


In [11]:
preferred_cities_df.count()

City_ID       55
City          55
Country       55
Date          55
Lat           55
Lng           55
Max Temp      55
Humidity      55
Cloudiness    55
Wind Speed    55
dtype: int64